In [6]:
import numpy as np
import s2cl2

from richmol import CartTensor, HyperCartTensor, HyperStates, QuadMom, RotStates
from richmol.asymtop import inertia_tensor

In [7]:
atom_masses = s2cl2.atom_masses
atom_labels = s2cl2.atom_labels

# inertia tensor
imom = inertia_tensor(atom_masses, s2cl2.atom_xyz)

# PAS rotation matrix
d, v = np.linalg.eigh(imom)
pas = v.T

# rotate Cartesian tensors to PAS frame
dip_mol = pas @ np.array(s2cl2.dip_mol)
efg_mol_cl1 = pas @ np.array(s2cl2.efg_mol_cl1) @ pas.T
efg_mol_cl2 = pas @ np.array(s2cl2.efg_mol_cl2) @ pas.T

print("dipole moment x, y, z:", s2cl2.dip_mol)
print("dipole moment a, b, c:", dip_mol)

dipole moment x, y, z: [0.0, 0.0, 0.3647]
dipole moment a, b, c: [9.79206719e-12 3.64700000e-01 2.46334809e-11]


In [8]:
inp = (
    "c2",
    "A/MHz",      5533.8964,
    "B/MHz",      1393.8436,
    "C/MHz",      1232.6728,
    "DeltaJ/kHz",    0.556,
    "DeltaJK/kHz",  -5.115,
    "DeltaK/kHz",   24.70,
    "d1/kHz",        0.144
)

# rotational states
jmax = 10
states = RotStates.watson(jmax, inp, print_enr=True)


Compute rotational solutions using Watson's effective Hamiltonian approach
Input rotational constants (MHz):
          A  5533.896400000000 
          B  1393.843600000000 
          C  1232.672800000000 
     DeltaJ     0.000556000000 
    DeltaJK    -0.005115000000 
     DeltaK     0.024700000000 
         d1     0.000144000000 
Watson reduction form: S
Symmetry group: c2
axes convention: a -> z (near-prolate)
solve for J = 0 and symmetry A ...
solve for J = 0 and symmetry B ...
axes convention: a -> z (near-prolate)
solve for J = 1 and symmetry A ...
solve for J = 1 and symmetry B ...
axes convention: a -> z (near-prolate)
solve for J = 2 and symmetry A ...
solve for J = 2 and symmetry B ...
axes convention: a -> z (near-prolate)
solve for J = 3 and symmetry A ...
solve for J = 3 and symmetry B ...
axes convention: a -> z (near-prolate)
solve for J = 4 and symmetry A ...
solve for J = 4 and symmetry B ...
axes convention: a -> z (near-prolate)
solve for J = 5 and symmetry A ...
sol

In [9]:
# since quantization z-axis corresponds to a-axis,
# swap axes such that abc->zyx, i.e. permute (13)

perm_mat = np.array([[0, 0, 1], [0, 1, 0], [1, 0, 0]])

dip_lab = CartTensor(states, perm_mat @ dip_mol)
efg_lab_cl1 = CartTensor(states, perm_mat @ efg_mol_cl1 @ perm_mat.T)
efg_lab_cl2 = CartTensor(states, perm_mat @ efg_mol_cl2 @ perm_mat.T)

In [10]:
# Define symmetry coupling rules between rotational and nuclear spin states.
# Each Cl nucleus has spin I = 3/2, so the total nuclear spin (I_total) can be 0, 1, 2, or 3.
# The corresponding nuclear spin symmetries in C2 group are B, A, B, and A, respectively.
#
# Due to the Pauli exclusion principle, the total spin-rotational wavefunction must be
# antisymmetric (B symmetry) under exchange of the two identical fermionic Cl nuclei.
#
# As a result:
# - Nuclear spin states with I_total = 0 and 2 (B symmetry) must couple to rotational states with A symmetry.
# - Nuclear spin states with I_total = 1 and 3 (A symmetry) must couple to rotational states with B symmetry.
#
# The function `symmetry_rules` below encodes these selection rules.


def symmetry_rules(
    j_list: list[int],
    j_sym_list: dict[int, list[str]],
    spin_list: list[tuple[float]],
) -> dict[str, list[tuple[int, tuple[float], str, str]]]:
    """Generates allowed spin-rotational symmetry combinations based
    on rotational and spin quantum numbers and symmetry labels.

    Returns a dictionary mapping resulting total (spin-rotational) symmetry
    labels to the corresponding combinations of rotational and spin quantum
    numbers and symmetries that yield them.

    Args:
        j_list (list[int]):
            List of rotational angular momentum quantum numbers J.

        j_sym_list (dict[int, list[str]]):
            Dictionary containing list of rotational symmetry labels for each J.

        spin_list (list[tuple[float]]):
            List of spin quantum numbers, each represented as a tuple
            (containing intermediate spin quanta).
            The total spin is given by the last value in each tuple.

    Returns:
        dict[str, list[tuple[int, tuple[float], str, str]]]:
            A dictionary where each key is a spin-rotational symmetry label (str),
            and each value is a list of tuples of the form:
                (J, spin_tuple, rotational_symmetry, spin_symmetry)
            representing the rotational and spin state components whose direct product
            yields the given key symmetry.

    Example:
        {
            'B': [
                (1, (1.5, 0), "A", "B"),
                (1, (1.5, 1), "B", "A"),
                (1, (1.5, 2), "A", "B"),
                ...
            ],
            ...
        }
    """
    c2_prod = {
        ("A", "B"): "B",
        ("B", "A"): "B",
        # ("A", "A"): "A",  # forbidden by Pauli principle
        # ("B", "B"): "A",  # forbidden by Pauli principle
    }
    j_spin_list = {"B": []}
    for j, spin in zip(j_list, spin_list):
        tot_spin = spin[-1]  # can be 0, 1, 2, 3
        for j_sym in j_sym_list[j]:
            if tot_spin in (1, 3):
                spin_sym = "A"
            elif tot_spin in (0, 2):
                spin_sym = "B"
            else:
                raise ValueError(
                    f"Symmetry of state with total spin = {tot_spin} is not defined"
                )
            try:
                f_sym = c2_prod[(j_sym, spin_sym)]
            except KeyError:
                continue
            j_spin_list[f_sym].append((j, spin, j_sym, spin_sym))
    return j_spin_list

In [11]:
# Compute hyperfine states and matrix elements of laboratory-frame tensors

# quadrupole moments (in mb) from Pyykkö, "Year-2008 nuclear quadrupole moments",
#   Mol. Phys. 106 (2008) 1965, http://dx.doi.org/10.1080/00268970802018367
quad_cl35 = QuadMom(spin=3 / 2, Q=-81.65)
quad_cl37 = QuadMom(spin=3 / 2, Q=-64.35)

hyper_states = HyperStates(
    0,
    7.0,
    states,
    spin_op=[quad_cl35, quad_cl35],
    efg_op=[efg_lab_cl1, efg_lab_cl2],
    symmetry_rules=symmetry_rules,
)

hyper_dip_lab = HyperCartTensor(hyper_states, dip_lab)


Compute hyperfine states
List of F quanta: [np.float64(0.0), np.float64(1.0), np.float64(2.0), np.float64(3.0), np.float64(4.0), np.float64(5.0), np.float64(6.0), np.float64(7.0)]
F   tot.sym.   J   (I_1, I_12, ... I_1N)     rovib.sym.   spin.sym.  rovib.dim 
----------------------------------------------------------------------------
0.0 B          0   (1.5, 0)                  A            B          1         
0.0 B          1   (1.5, 1)                  B            A          2         
0.0 B          2   (1.5, 2)                  A            B          3         
0.0 B          3   (1.5, 3)                  B            A          4         
1.0 B          1   (1.5, 0)                  A            B          1         
1.0 B          1   (1.5, 1)                  B            A          2         
1.0 B          2   (1.5, 1)                  B            A          2         
1.0 B          1   (1.5, 2)                  A            B          1         
1.0 B          2   (1.

In [12]:
for f, vec_f in hyper_states.vec.items():
    for sym, vec_sym in vec_f.items():
        # quanta = hyper_states.quanta_dict_k[f][sym]
        op_quanta = hyper_states.quanta_dict_op[f][sym]

        for istate in range(vec_sym.shape[-1]):
            print(f, sym, istate, op_quanta[istate])

0.0 B 0 [(0.0, 0.9999999999999991), (2.0, 5.728383419790729e-16), (1.0, 1.0977629970362595e-29), (3.0, 1.9343906945736396e-31)]
0.0 B 1 [(1.0, 1.0), (2.0, 5.946486636270781e-16), (3.0, 7.769104417754504e-17), (0.0, 1.3909225363737456e-29)]
0.0 B 2 [(1.0, 0.999999999999998), (2.0, 2.474539663893702e-15), (3.0, 2.2146152002602664e-17), (0.0, 2.314552654368934e-45)]
0.0 B 3 [(2.0, 0.9999999999999993), (3.0, 6.00418053858876e-16), (0.0, 3.2125194969858945e-16), (1.0, 2.8776988246252478e-18)]
0.0 B 4 [(2.0, 0.9999999999999969), (1.0, 2.4745396774511776e-15), (3.0, 1.7183399343909829e-15), (0.0, 4.1529855292265836e-36)]
0.0 B 5 [(3.0, 0.9999999999999998), (2.0, 8.971294538757653e-16), (1.0, 4.429081098003976e-17), (0.0, 5.076337578390561e-31)]
0.0 B 6 [(3.0, 0.9999999999999978), (2.0, 1.611024781111688e-15), (1.0, 8.006065773993519e-19), (0.0, 7.94353758410353e-50)]
0.0 B 7 [(2.0, 0.9999999999999984), (3.0, 8.919597330375042e-16), (1.0, 5.917709824420155e-16), (0.0, 2.515863922804803e-16)]
0